<a href="https://colab.research.google.com/github/JayGitProfile/Calorie-prediction-xgboost-regressor/blob/main/Apriori%20trial%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PA 3: Association Analysis - Apriori/FP Growth

## Student Details 

**Student Name and ID: Jayakanth Srinivasan 1001919453

**Group members name and ID : 
Jayakanth Srinivasan 1001919453, Indu Katta 1001956615, Hari Puvvada 1001989465

## Submission Instructions

Rename this submission file as specified in the instructions document

## Tasks

Imports

In [66]:
!pip install apriori
!pip install mlxtend
!pip install fpgrowth
!pip install apyori
%pip install mlxtend --upgrade
%pip install apriori --upgrade

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from apyori import apriori
from sklearn.preprocessing import MultiLabelBinarizer

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1.    Load the dataset (from the csv file)

In [111]:
# Write your code here
df = pd.read_csv("dataset1.csv")

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.    Pre-process the dataset and provide a detailed explanation (such as Handling of null values, empty strings, unncessary trailing characters such as commas and white spaces)

Removing Tid column as it has no significance

In [112]:
df.drop('Tid', axis=1, inplace=True)
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items
0,"Bread, Corn, Dill, Ice cream, Sugar, Yogurt, c..."
1,"Milk,"
2,"Apple, Butter, Dill, Ice cream, Milk, Yogurt, ..."
3,"Butter, Cheese, Dill, Milk, Nutmeg, Onion,"
4,"Apple, Bread,"


Checking for null / empty values

In [113]:
print('Null/Empty values:\n',df.isna().sum())

Null/Empty values:
 items    0
dtype: int64


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Removing trailing whitespaces and commas

In [114]:
df['items'] = df['items'].str.strip(', ')
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items
0,"Bread, Corn, Dill, Ice cream, Sugar, Yogurt, c..."
1,Milk
2,"Apple, Butter, Dill, Ice cream, Milk, Yogurt, ..."
3,"Butter, Cheese, Dill, Milk, Nutmeg, Onion"
4,"Apple, Bread"


Removing all whitespaces

In [115]:
df['items'] = df['items'].str.replace(" ", "")
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items
0,"Bread,Corn,Dill,Icecream,Sugar,Yogurt,chocolate"
1,Milk
2,"Apple,Butter,Dill,Icecream,Milk,Yogurt,chocolate"
3,"Butter,Cheese,Dill,Milk,Nutmeg,Onion"
4,"Apple,Bread"


Converting data to lowercase for unified format

In [116]:
df['items'] = df['items'].str.lower()
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items
0,"bread,corn,dill,icecream,sugar,yogurt,chocolate"
1,milk
2,"apple,butter,dill,icecream,milk,yogurt,chocolate"
3,"butter,cheese,dill,milk,nutmeg,onion"
4,"apple,bread"


3.    Encode the dataset

Applying one hot encoding

In [74]:
df['items'] = df['items'].str.split(',')

multiLabelBinarizer = MultiLabelBinarizer()
encodedDF = pd.DataFrame(multiLabelBinarizer.fit_transform(df2['items']), 
                       columns=multiLabelBinarizer.classes_, index=df.index)

df = pd.concat([df, encodedDF], axis=1)
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items,apple,bread,butter,cheese,chocolate,corn,dill,eggs,icecream,kidneybeans,milk,nutmeg,onion,sugar,unicorn,yogurt
0,"[bread, corn, dill, icecream, sugar, yogurt, c...",0,1,0,0,1,1,1,0,1,0,0,0,0,1,0,1
1,[milk],0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,"[apple, butter, dill, icecream, milk, yogurt, ...",1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1
3,"[butter, cheese, dill, milk, nutmeg, onion]",0,0,1,1,0,0,1,0,0,0,1,1,1,0,0,0
4,"[apple, bread]",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
#dropping items column

df = df.drop(['items', 'unicorn'], axis=1)
df.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,apple,bread,butter,cheese,chocolate,corn,dill,eggs,icecream,kidneybeans,milk,nutmeg,onion,sugar,yogurt
0,0,1,0,0,1,1,1,0,1,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1
3,0,0,1,1,0,0,1,0,0,0,1,1,1,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


Checking for duplicates

In [76]:
print('Duplicates: ',df.duplicated().sum())

df.drop_duplicates(inplace=True)

Duplicates:  93


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


For each of the combinations in tasks 4 & 5, print the output in the following Sample format: 

- Support = 0.02
- Confidence = 0.1
- Total Number of Association Rules: 10
- Top 5 rules ordered by Confidence descending:
- Number of Maximal Sets: 5
- Number of Closed Sets: 5

4.	Execute Apriori algorithm and generate rules for 3 different combinations of support and confidence

In [ ]:
# Write your code here
for support, confidence in [(0.02, 0.5), (0.03, 0.6), (0.04, 0.7), (0.05, 0.6)]:
    results = list(apriori(df.values.tolist(), min_support=support, min_confidence=confidence))
    print(f"Support: {support}, Confidence: {confidence}")
    for result in results:
        print(result)

In [ ]:
params = [
    {'support': 0.002, 'confidence': 0.001},
    {'support': 0.003, 'confidence': 0.002},
    {'support': 0.004, 'confidence': 0.003},
    {'support': 0.005, 'confidence': 0.004}
]

# Loop through the list of parameters and execute the apriori algorithm for each combination
for i, param in enumerate(params):
    # Execute the apriori algorithm
    results = list(apriori(df.values, min_support=param['support'], min_confidence=param['confidence']))
    
    # Print the output in the desired format
    print(f"Combination {i+1}: Support = {param['support']}, Confidence = {param['confidence']}")
    print(f"Total Number of Association Rules: {len(results)}")
    
    # Sort the rules by confidence in descending order
    sorted_results = sorted(results, key=lambda x: x.ordered_statistics[0].confidence, reverse=True)
    
    # Print the top 5 rules ordered by confidence descending
    print("Top 5 rules ordered by Confidence descending:")
    for j, r in enumerate(sorted_results[:5]):
        print(f"{j+1}. {list(r.items)} => {list(r.ordered_statistics[0].items_base)} (Confidence: {r.ordered_statistics[0].confidence:.2f})")
    
    # Print the number of maximal and closed sets
    print(f"Number of Maximal Sets: {len(list(apriori(df.values, min_support=param['support'], min_confidence=param['confidence'], max_length=None)))}")
    print(f"Number of Closed Sets: {len(list(apriori(df.values, min_support=param['support'], min_confidence=param['confidence'], max_length=1)))}\n")

In [102]:
results = apriori(df.values.tolist(), min_support=0.5, min_confidence=0.5)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: ignored

In [94]:
results = list(apriori(df.values.tolist(), min_support=0.5))

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: ignored

In [93]:
rules = apriori(df.values.tolist(), min_support=0.2, min_confidence=0.5)

# Print the output
print(f"Support = {rules.support}")
print(f"Confidence = {rules.confidence}")
print(f"Total Number of Association Rules: {len(rules)}")

# Sort the rules by confidence
sorted_rules = sorted(rules, key=lambda rule: rule.confidence, reverse=True)

# Print the top 5 rules by confidence
print("Top 5 rules ordered by Confidence descending:")
for rule in sorted_rules[:5]:
    print(f"{rule.lhs} => {rule.rhs} (Confidence: {rule.confidence:.2f})")

# Print the number of maximal and closed sets
maximal_sets = sum(1 for rule in rules if len(rule.lhs) == 2 and rule.support == rules.support)
closed_sets = sum(1 for rule in rules if not any(rule <= r for r in rules if r != rule))
print(f"Number of Maximal Sets: {maximal_sets}")
print(f"Number of Closed Sets: {closed_sets}")

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: ignored

5.    Execute FP Growth algorithm and generate rules for 3 different combinations of support and confidence

In [79]:
# Write your code here


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6.	Plot the following graphs comparing both the algorithms:
    - time vs minimum support
    - time vs number of itemsets

In [80]:
# Write your code here


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


7.    Explain the insights that you got from the graphs. Which algorithm worked better and what do you believe is the reason?

In [81]:
# Write your code here


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Rubric
* [02 points] - Student Details and complying with submission instructions 
* [03 points] - Task 1 
* [10 points] - Task 2
* [05 points] - Task 3
* [25 points] - Task 4
* [25 points] - Task 5
* [20 points] - task 6
* [10 points] - task 7

In [117]:
df2 = df
df2['items'] = df2['items'].str.split(',')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [118]:
df2.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,items
0,"[bread, corn, dill, icecream, sugar, yogurt, c..."
1,[milk]
2,"[apple, butter, dill, icecream, milk, yogurt, ..."
3,"[butter, cheese, dill, milk, nutmeg, onion]"
4,"[apple, bread]"


In [ ]:
records = []
for i in range(len(df2)):
  records.append(df.values[i])\

records

In [139]:
my_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
my_list2 = [[1, 0, 0], [0, 0, 1], [1, 1, 1]]
my_list3 = [['aa', 'bb', 'cc'], ['aa', 'cc', 'dd'], ['dd', 'aa', 'bb']]

rules = apriori(my_list, min_support=0.0022, min_confidence=0.20, min_lift=3, min_length = 4)

# Store rules in result variable
results = list(rules)
# See the items that were bought together with their support
results_list = []
for i in range(0, len(results)):
    results_list.append('RULE:\t' + str(results[i][0]) + '\nSUPPORT:\t' + str(results[i][1]))

print(len(results_list))

for i in range(len(results_list)):
  print(results_list[i])

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
